<a href="https://colab.research.google.com/github/dymiyata/intro-to-ml-and-ai-2025-2026/blob/main/decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a Decision Tree Classifier

First we import some of the usual stuff.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Dataset

For this example, we will use seaborn's built in `"titanic"` dataset. We can load the dataset and save it to a dataframe using `sns.load_dataset()`.

In [5]:
df = sns.load_dataset("titanic")

Let's get an idea of what this dataset contains.

In [19]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


For this example, let's predict whether a given passenger survived or not.  There are many features here, so to simplify things, let's just go with four features: `"sex"`, `"class"`, `"age"`, `"fare"`, `"survived"`.

When we filter, be sure to use `.dropna` to get rid of any entries with missing values.

In [20]:
df_filtered = df[["sex", "pclass", "age", "fare", "survived"]]

Now let's define our feature matrix `X` and target vector `y` using this data.

In [25]:
X = df_filtered[["sex", "pclass", "age", "fare"]]
y = df_filtered["survived"]

### Trying to Run the Decision Tree Classifier

Now we import the model and try to fit it to the data:

- For linear and logisitic regression, we had to import `LinearRegression` or `LogisticRegression` from `sklearn.linear_model`.  

- To make a decision tree, we have to import `DecisionTreeClassifier` from `sklearn.tree`.

In [15]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

Let's fit it to the data:

Looks like we got an error...

### One Hot Encoding


We got an error because every feature for `sklearn`'s `DecisionTreeClassifier` (even the categorical ones) must be either:
- numerical
- Something Python knows how to automatically convert to numbers
  - Like `True`/`False` which will be converted to `1` or `0`

Currently, the `"sex"` feature is not of this form.  Python doesn't know how to convert `"male"` and `"female"` to numbers.


This is where one hot encoding comes in.  Recall one hot encoding does the following:
- Take each *value* of a categorical variable (e.g. `"male"` is a value of the variable `"sex"`)
- Make a new feature just for that value.
- Examples with that value get a `1` for the new feature
- Examples with a different value get a `0` for the new feature

To do this, we run `pd.get_dummies` on our feature matrix and store the result in another variable.  Let's see what happens when we do this:

In [26]:
X_encoded = pd.get_dummies(X)

But notice, these two new features are a bit redundant:
- If a passenger is `"male"` you know they aren't `"female"` and vice versa.

Thus, we can get rid of one of these new features from the one-hot-encoding. To do this automatically, we add the argument `drop_first = True`.

In [28]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [29]:
X_encoded.head()

,pclass,age,fare,sex_male
0,3,22.0,7.2500,True
1,1,38.0,71.2833,False
2,3,26.0,7.9250,False
3,1,35.0,53.1000,False
4,3,35.0,8.0500,True


### Fitting the model

Now that we've done the one-hot-encoding, we can actually fit the model.  First, let's do the correct practice of splitting the data into a training set and a testing set. (we need to import `train_test_split` from `sklearn.model_selection` first)
  - Don't forget we should stratify by y with any classification problem

Now fit the model on the training data:

To evaluate our model we make predictions on the test set:

We can use the confusion matrix or classification report to see how well we did:

In [95]:
from sklearn.metrics import confusion_matrix, classification_report

Our model seems to be overfitting the training data...

This could be because the default stopping condition for sklearn's decision tree classifier is to keep going until the gini impurity stops decreasing.

Let's look at the depth of our tree. We can do this with `model.get_depth()`

Let's try another decision tree where we set the max depth of the tree.

The max depth is called a *hyperparameter* since it is not part of the actual model's "formula", but it is a value that affects the training of the model.

Thus, our act of adjusting things like max depth is called *hyperparameter tuning* and is an important aspect of machine learning which we will continue to talk more about in the future.